In [270]:
#! nvidia-smi

In [271]:
#conda install pytorch torchvision -c pytorch

In [272]:
import os
def EXIT_NOTEBOOK(): os._exit(00)
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [273]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
import torch 
from torch import nn
import torch.nn.functional as func
import seaborn as sns
import matplotlib.ticker

# # ps
# import pysindy as ps

# sns.set_theme()
torch.set_default_dtype(torch.float64)
plt.rcParams['text.usetex'] = True

In [274]:
dim = 6
torch.set_default_dtype(torch.float32)

# Dataset for X

In [275]:
from BeadModel import Simulate
from SimulationParameters import *
X = Simulation(*Simulate(numSims)).positions[:,:,:,0]
print(X.shape)
print(X)

(3, 2, 11)
[[[-1.85958495e-02 -2.34382707e-02 -5.42650682e-03 -2.39646514e-02
   -3.77401638e-02 -6.45540354e-02 -9.54499720e-02 -9.50182513e-02
   -6.45613912e-02 -8.95568240e-02 -8.95568240e-02]
  [-1.84951965e-02 -4.40069403e-02 -4.67088782e-02 -4.57419627e-02
   -5.89102125e-02 -2.93702734e-02 -1.20002004e-02 -2.51019380e-02
   -3.79777379e-02 -4.85348623e-02 -4.85348623e-02]]

 [[-1.11380522e-02 -2.28152367e-02 -3.56737015e-02 -2.46125531e-02
   -2.72768969e-03 -2.03515907e-02 -2.40362412e-02 -3.89467749e-02
   -3.81666208e-02 -2.38841518e-02 -2.38841518e-02]
  [-4.84693304e-03 -7.65963959e-03 -1.04494194e-02 -1.40798705e-02
   -8.88022471e-03 -2.46445275e-04  1.35308888e-02  9.90989935e-03
   -2.03964478e-02 -3.86028364e-03 -3.86028364e-03]]

 [[-9.24629130e-03 -1.60675901e-02 -1.84644186e-02 -4.14103594e-03
    1.41646522e-03 -3.66584557e-02 -3.86298577e-02 -1.99072503e-02
   -1.61804482e-02 -1.06451244e-02 -1.06451244e-02]
  [-3.47521240e-02 -7.79033292e-03 -8.01978745e-03  1.3

# Set the NN model and Solver with training process

In [276]:
def relu2(X): return func.relu(X)**2
def tanh(X): return func.tanh(X)
class FCNN(nn.Module):
    def __init__(self,input_dim=6,output_dim=6,num_hidden=2,hidden_dim=10,act=func.tanh,transform=None):
        super().__init__()
         
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers  = nn.ModuleList([nn.Linear(input_dim,hidden_dim)])
        for _ in range(num_hidden-1): self.layers.append(nn.Linear(hidden_dim,hidden_dim))
        self.act     = act
        self.out     = nn.Linear(hidden_dim,output_dim)
        self.transform = transform
    def forward(self,X):
        if self.transform is not None: X = self.transform(X)
        for layer in self.layers: X = self.act(layer(X))
        Y = self.out(X)
        return Y
class Model(nn.Module):
    def __init__(self,dim,model_U,unit_len=int(5e3)):
        super().__init__()
        self.dim      = dim
        self.model_U  = model_U
        self.unit_len = unit_len
        self.mu       = nn.Parameter(torch.tensor([0.]*dim),requires_grad=False) 
        self.sigma    = nn.Parameter(torch.tensor([1.]*dim),requires_grad=False)
        #above two lines should work, but if something doesn't work, check here! Does current self.mu and self.sigma code work if dim>1. Should return a vector since what we're trying to do is calculate mu
        #and sigma separately for each imnputted feature. mu is parameter[0] for each row.
        self.coef_U   = nn.Parameter(torch.tensor(1.)*dim,requires_grad=False)
        #self.mu       = nn.Parameter(torch.tensor([0.]*dim).cuda(),requires_grad=False)
        #self.sigma    = nn.Parameter(torch.tensor([1.]*dim).cuda(),requires_grad=False)
        #self.coef_U   = nn.Parameter(torch.tensor(1.).cuda(),requires_grad=False)
    def get_U_harmonic(self,X): return torch.sum(X**2,axis=-1)


    def get_U_dU(self, X):
        U_list = []
        dU_list = []
        for t in range(X.shape[-1]):
            X_t = X[..., t]
            X_t_ = torch.tensor(X_t, dtype=torch.float32, requires_grad=True)
            # X_t_ = (X_t - self.mu) / self.sigma #normalization necessary, column wise is standard
            # U_t = torch.sum(self.coef_U * (self.model_U(X_t_) + self.get_U_harmonic(X_t_))) #*certify validity
            print()
            print(self.coef_U)
            print()
            print(self.model_U(X_t_))
            print()
            print(self.get_U_harmonic(X_t_))
            print()
            U_t = self.coef_U * (self.model_U(X_t_).view(-1) + self.get_U_harmonic(X_t_))
            U_list.append(U_t)
            dU_t = torch.autograd.grad(
                outputs=U_t,
                inputs=X_t,
                grad_outputs=torch.ones_like(U_t),
                create_graph=True
            )[0]
            dU_list.append(dU_t)
        U = torch.stack(U_list, dim=0)
        dU = torch.stack(dU_list, dim=-1)
        return U, dU

    
    def get_U_np(self,X): 
        U,_ = self.get_U_dU(X);
        return U.cpu().data.numpy()
    
class Solver():
    def __init__(self,model):
        self.model=model
    def train_model(self,data_train,data_test,get_loss,optimizer,
                    n_steps,batch_size,scheduler=None,n_show_loss=100,error_model=None,use_tqdm=True):
        if use_tqdm: step_range = tqdm(range(n_steps))
        else: step_range = range(n_steps)
        loss_step = []
        for i_step in step_range:
            if i_step%n_show_loss==0:
                loss_train,loss_test = get_loss(self.model,data_train)[:-1],\
                                       get_loss(self.model,data_test)[:-1]
                
                def show_num(x): 
                    if abs(x)<100 and abs(x)>.01: return '%0.5f'%x
                    else: return '%0.2e'%x
                item1 = '%2dk'%np.int_(i_step/1000)
                item2 = 'Loss: '+' '.join([show_num(k) for k in loss_train])
                item3 = ' '.join([show_num(k) for k in loss_test])
                item4 = ''
                if error_model is not None: item4 = 'E(QP): %0.4f' % (error_model(self.model))
                print(', '.join([item1,item2,item3,item4]))
                loss_step = loss_step + [i_step] + [k.cpu().data.numpy() for k in loss_train]\
                                                 + [k.cpu().data.numpy() for k in loss_train]
            data_batch = data_train[random.sample(range(len(data_train)),
                                                  min(batch_size,len(data_train)))]
#             print(i_step,data_batch.shape)
            loss = get_loss(self.model,data_batch)[-1]
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if scheduler is not None: scheduler.step()
        if error_model is not None: 
            print("Error: %0.5f" % (error_model(self.model)))
        return loss_step

In [277]:
model_U = FCNN(input_dim=dim,output_dim=1,num_hidden=3,hidden_dim=10,act=tanh)#.cuda()
model   = Model(dim,model_U=model_U)#.cuda();
SOL     = Solver(model)

In [278]:
# model.get_U_dU(X[:3])

In [279]:
# print(model.mu.shape)
# print(model.mu)

# Set the loss function and Train the model for differen a_k(x)

In [280]:
from Loss import getResidue

# def plot_model(model,cmap='terrain',max_V = 10):
    
#     xx     = np.linspace(0,2,1000).reshape(-1,1)
#     U_NN   = model.get_U_np(xx)
#     U_NN_min = U_NN.min()
#     U_NN  = U_NN-U_NN_min

#     fig, ax    = plt.subplots(1,1,figsize=(5,3),dpi=200,constrained_layout=True)
#     c      = ax.plot(xx[:,0],U_NN,'-',lw=1.5,)

#     ax.tick_params(axis="both", labelsize=10)
#     plt.show()
#def get_a(X,k=choose_id):
#    if choose_id==1: return 2*torch.exp(-3*X**2)
#    if choose_id==2: return 2/(1+torch.exp(20*(torch.abs(X)-0.75)))
#    if choose_id==3: return 4/(1+torch.exp(20*(torch.abs(X)-0.75)))
def get_loss(model,data):
    X = data
    X = torch.tensor(X,requires_grad=True)#.cuda()
    _,dU = model.get_U_dU(X)

    loss = getResidue(X, dU)
    return loss#,loss

# plot_model(model)
get_loss(model,X[:3])


Parameter containing:
tensor(6.)



/var/folders/jf/w5m4c3sj361fdh8b5wz1cy1m0000gn/T/ipykernel_97939/3909905085.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_t_ = torch.tensor(X_t, dtype=torch.float32, requires_grad=True)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x2 and 6x10)

In [84]:
#for choose_id in [1,2,3]:
model_U = FCNN(input_dim=dim,output_dim=1,num_hidden=3,hidden_dim=10,act=tanh)#.cuda()
model   = Model(dim,model_U=model_U)#.cuda();
SOL     = Solver(model)

print(model.mu,model.sigma,model.coef_U)
    #optimizer = torch.optim.Adam(model.parameters(), lr=torch.tensor(0.001).cuda())
optimizer = torch.optim.Adam(model.parameters(), lr=torch.tensor(0.001))
scheduler = None
_loss_step = SOL.train_model(data_train=X,data_test=X,
                             get_loss=get_loss,optimizer=optimizer,scheduler=scheduler,
                             n_steps=int(5e4+1),batch_size=500,n_show_loss=1000,use_tqdm=True)
torch.cuda.empty_cache()
plot_model(model)
   # torch.save(model.state_dict(), "savee/model_"+str(choose_id))
#torch.save(model.state_dict(),"savee/model_anaconda3")

<bound method Module.type of FCNN(
  (layers): ModuleList(
    (0): Linear(in_features=6, out_features=10, bias=True)
    (1-2): 2 x Linear(in_features=10, out_features=10, bias=True)
  )
  (out): Linear(in_features=10, out_features=1, bias=True)
)>
Parameter containing:
tensor([0., 0., 0., 0., 0., 0.]) Parameter containing:
tensor([1., 1., 1., 1., 1., 1.]) Parameter containing:
tensor(1.)


  0%|          | 0/50001 [00:00<?, ?it/s]

/var/folders/jf/w5m4c3sj361fdh8b5wz1cy1m0000gn/T/ipykernel_89714/972852114.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_t_ = torch.tensor(X_t, dtype=torch.float32, requires_grad=True)


RuntimeError: mat1 and mat2 must have the same dtype, but got Float and Double

In [108]:
#torch.save(model.state_dict(), "savee/model_99")
#model.state_dict()

In [ ]:
fig, ax  = plt.subplots(1,1,figsize=(4,3),dpi=200,constrained_layout=True)
xx       = np.linspace(0,2,1000).reshape(-1,1)
U_NN     = model.get_U_np(xx)
U_NN_min = U_NN.min()
U_NN     = U_NN-U_NN_min
c        = ax.plot(xx[:,0],U_NN,'-',lw=1.5)
ax.legend(fontsize=10)
ax.set_xlabel('$x$',fontsize=10)
ax.set_ylabel('$U(x)$',fontsize=10)
ax.set_xlim([0,2])
ax.set_yticks([0,0.2,0.4,0.6,0.8,1,1.2])
ax.set_xticks([0,.5,1.,1.5,2])
ax.yaxis.grid(linestyle='--')
ax.tick_params(axis="both", labelsize=10)
plt.show()

In [ ]:
U_NN

In [30]:
import scipy.io as scio
from scipy.io import savemat


# Visualizing the results for different a_k(x)

In [ ]:
def plot_models(models):
    
    xx       = np.linspace(0,2,1000).reshape(-1,1)
    fig, ax  = plt.subplots(1,1,figsize=(4,3),dpi=200,constrained_layout=True)
    
    for k,model_name in enumerate(models):
        model.load_state_dict(torch.load(model_name))
        U_NN     = model.get_U_np(xx)
        U_NN_min = U_NN.min()
        U_NN     = U_NN-U_NN_min
        c        = ax.plot(xx[:,0],U_NN,'-',lw=1.5,label="$a_{%d}(x)$"%(k+1))
    ax.legend(fontsize=10)
    ax.set_xlabel('$x$',fontsize=10)
    ax.set_ylabel('$U(x)$',fontsize=10)
    ax.set_xlim([0,2])
    ax.set_yticks([0,0.2,0.4,0.6,0.8,1,1.2])
    ax.set_xticks([0,.5,1.,1.5,2])
    ax.yaxis.grid(linestyle='--')
    ax.tick_params(axis="both", labelsize=10)
    plt.show()
    
plot_models(["savee/model_anaconda3"])
#plot_models(["savee/model_a1_update", "savee/model_a2_update", "savee/model_a3_update", "savee/model_a4", "savee/model_a5_update"])

In [ ]:
model

In [ ]:
torch.load("/Users/annacoletti/Desktop/savee/model_a3_update.mat")